In [1]:
import pandas as pd
from os.path import join

In [4]:
data_path = "/home/akash/Documents/AML-Hackathon/Problem-1/dataset.csv" #join("..", "..", "Dataset-1", "selfie_dataset.txt")
image_path = "/home/akash/Documents/dataset/" #join("..", "..", "Dataset-1", "selfie_dataset.txt")#join("..", "..", "Dataset-1", "images")

In [6]:
headers = [
    "image_name", "score", "partial_faces" ,"is_female" ,"baby" ,"child" ,"teenager" ,"youth" ,"middle_age" ,"senior" ,"white" ,"black" ,"asian" ,"oval_face" ,"round_face" ,"heart_face" ,"smiling" ,"mouth_open" ,"frowning" ,"wearing_glasses" ,"wearing_sunglasses" ,"wearing_lipstick" ,"tongue_out" ,"duck_face" ,"black_hair" ,"blond_hair" ,"brown_hair" ,"red_hair" ,"curly_hair" ,"straight_hair" ,"braid_hair" ,"showing_cellphone" ,"using_earphone" ,"using_mirror", "braces" ,"wearing_hat" ,"harsh_lighting", "dim_lighting"
]
df_image_details = pd.read_csv(data_path, names=headers, header=None)
df_image_details.head(3)

,image_name,score,partial_faces,is_female,baby,child,teenager,youth,middle_age,senior,...,curly_hair,straight_hair,braid_hair,showing_cellphone,using_earphone,using_mirror,braces,wearing_hat,harsh_lighting,dim_lighting
0,10175354_375525535919416_849604953_a,4.518,-1,1,-1,-1,-1,1,-1,-1,...,1,-1,-1,1,-1,1,-1,-1,-1,-1
1,10013197_1488786631335003_762671051_a,4.025,-1,1,-1,-1,-1,1,-1,-1,...,-1,1,-1,1,-1,1,-1,-1,-1,-1
2,924319_592681537494843_2039700997_a,4.695,-1,1,-1,-1,-1,1,-1,-1,...,-1,1,-1,-1,-1,-1,-1,-1,-1,-1


In [7]:
needed_columns = [
    'image_name',
    'is_female',
    'baby',
    'child',
    'teenager',
    'youth',
    'middle_age',
    'senior'
]

In [8]:
df_image_details = df_image_details[needed_columns]
df_image_details = df_image_details[df_image_details.is_female != 0]
df_image_details.replace(to_replace=-1, value=0, inplace=True)

In [9]:
image_names = df_image_details.image_name.values.copy()
image_attrs = df_image_details[needed_columns[1:]].values.copy()

In [10]:
image_paths = [join(image_path, iname) + '.jpg' for iname in image_names]

In [11]:
from keras.utils import Sequence
import numpy as np
import cv2

/home/akash/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [12]:
cv2.__version__

'3.4.3'

In [59]:
class ImageGenerator(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]

        # read your data here using the batch lists, batch_x and batch_y
        x = [self.read_image(filename) for filename in batch_x] 
        y = [atrributes for atrributes in batch_y]
        return np.array(x), np.array(y)
    
    def read_image(self, fname):
        print(fname)
        im = cv2.imread(fname)
        if im.any()!= None:
            im = cv2.resize(im, (224, 224))
        return im;

# Training Model

In [60]:
from keras.applications import resnet50
from keras.layers import Dense
from keras.models import Model, load_model

In [61]:
model_rnet = load_model("resnet50.hdf5")

/home/akash/anaconda3/lib/python3.6/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [62]:
model_rnet.layers.pop()

In [63]:
model_output = Dense(7, activation='softmax')(model_rnet.output)

In [64]:
model = Model(inputs=[model_rnet.input], outputs=[model_output])

In [65]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [66]:
train_gen = ImageGenerator(image_paths, image_attrs, batch_size=16)
test_gen = ImageGenerator(image_paths, image_attrs, batch_size=16)

In [67]:
model.fit_generator(train_gen, epochs=200, steps_per_epoch=16)

Epoch 1/200
/home/akash/Documents/dataset/927263_1406733892930614_404232657_a.jpg
/home/akash/Documents/dataset/1172224_851053758254060_195181990_a.jpg


AttributeError: 'NoneType' object has no attribute 'any'

/home/akash/Documents/dataset/1172233_390705051071557_412638630_a.jpg
/home/akash/Documents/dataset/1172233_566300846800644_408527685_a.jpg
/home/akash/Documents/dataset/1172236_273253419521553_1046206723_a.jpg
/home/akash/Documents/dataset/1173152_564703873584320_220719933_a.jpg
/home/akash/Documents/dataset/1173157_128541823983220_239211292_a.jpg
/home/akash/Documents/dataset/1208275_556709621076379_1427962805_a.jpg
/home/akash/Documents/dataset/1208374_1441286662776787_1388172366_a.jpg
/home/akash/Documents/dataset/1208396_621892897893318_411578799_a.jpg
/home/akash/Documents/dataset/1208399_664560933603695_214983266_a.jpg
/home/akash/Documents/dataset/1208399_836206343059893_741216869_a.jpg
/home/akash/Documents/dataset/1208402_301920106632621_1610842149_a.jpg
/home/akash/Documents/dataset/1208426_756457577749736_1741380949_a.jpg
/home/akash/Documents/dataset/1208429_1462695360631004_1711729509_a.jpg
/home/akash/Documents/dataset/1208445_635654069833071_2118767743_a.jpg
/home/akash

In [23]:
model.save('temp.hdf5')